In [1]:
import torch
import torch.nn as nn
#import pandas as pd
from HGP.models import Model
import pickle
import yaml
from pathlib import Path
from sklearn.model_selection import train_test_split
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader
import numpy as np

In [2]:
import networkx as nx

In [3]:

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.profilers import SimpleProfiler
from torch_geometric.datasets import TUDataset

# BITTE EINMAL DEN MODELL NAMEN HIER EINMAL EINTRAGEN (HGP-SL o.ä.)

Damit die Datei unter dem Namen gespeichert wird

In [4]:
model_name = "emp-nci"

In [5]:
def yaml_dump_for_notebook(filepath="configs/baseline.yml"):
    # Call this function if you use this as a notebook so bypass argparse and directly dump the config
    with open(str(filepath), "r", encoding="utf-8") as stream:
        args_dict = yaml.safe_load(stream) or dict()

    return args_dict

In [6]:
# %%
args_dict = yaml_dump_for_notebook("emp-gcn-nci-grid001.yml")

pl.seed_everything(args_dict["random_seed"], workers=True)


Global seed set to 21


21

In [7]:
# %%
# Load the dataset
dataset = TUDataset(
    Path.cwd().joinpath("datasets"),
    name=args_dict["dataset"],
    use_node_attr=True,
)
args_dict["num_classes"] = dataset.num_classes
args_dict["num_features"] = dataset.num_features

In [8]:
split_train = int(dataset.len() * args_dict["split_ratio"])
split_val = int((dataset.len() - split_train) * args_dict["test_ratio"])
split_test = dataset.len() - split_val - split_train

print(
    f"Hole: {dataset.len()}, Train: {split_train}, Val: {split_val}, Test: {split_test}"
)
# %%
train_data, eval_data, test_data = random_split(
    dataset,
    [split_train, split_val, split_test],
    generator=torch.Generator().manual_seed(777),
)

Hole: 4110, Train: 2055, Val: 1027, Test: 1028


train_data, eval_data, test_data = random_split(
    dataset,
    [split_train, split_val, split_test],
    generator=torch.Generator().manual_seed(42),
)

In [9]:
# %%
train_loader = DataLoader(
    train_data,
    batch_size=args_dict["batch_size"],
    shuffle=False,
    num_workers=args_dict["num_workers"],
)
val_loader = DataLoader(
    eval_data,
    batch_size=args_dict["batch_size"],
    shuffle=False,
    num_workers=args_dict["num_workers"],
)
test_loader = DataLoader(
    test_data,
    batch_size= 1 ,# dataset.len() - split_test,
    shuffle=False,
    num_workers=args_dict["num_workers"],
)

In [10]:
# Define the model
from HGP.models import LightModel


model = LightModel(args_dict).to(args_dict["device"])
# load a saved model
weights = torch.load("gcn_model_0.pt")
# set the weights
model.load_state_dict(weights)


model.eval()

loss = nn.CrossEntropyLoss(reduction="none")
softmax = nn.Softmax(dim=1)

In [12]:

right_y = []
wrong_y = []

right_graphs = []
wrong_graphs = []

for batch in test_loader:
    data = batch.to(args_dict["device"])
    out = model(data)
    prediction_truth = np.argmax(
        softmax(out).detach().cpu().numpy()) == data.y.detach().cpu().numpy()
    break
    if prediction_truth[0]:
        g = nx.from_edgelist(data.edge_index.detach().cpu().numpy().T)
        right_graphs.append(g)
        right_y.append(data.y.detach().cpu().numpy()[0])
    else:
        g = nx.from_edgelist(data.edge_index.detach().cpu().numpy().T)
        wrong_graphs.append(g)
        wrong_y.append(data.y.detach().cpu().numpy()[0])

np_right_graphs = np.array(right_graphs, dtype=object)
np_wrong_graphs = np.array(wrong_graphs, dtype=object)
np_right_y = np.array(right_y)
np_wrong_y = np.array(wrong_y)

In [13]:
prediction_truth

array([False])

In [ ]:
# save numpy arrays to folder analysis
np.save(f"Error_Analysis/graphs/GCN/{args_dict['dataset']}_{model_name}_right_graphs.npy", np_right_graphs)
np.save(
    f"Error_Analysis/graphs/GCN/{args_dict['dataset']}_{model_name}_wrong_graphs.npy", np_wrong_graphs)
np.save(
    f"Error_Analysis/graphs/GCN/{args_dict['dataset']}_{model_name}_right_y.npy", np_right_y)
np.save(
    f"Error_Analysis/graphs/GCN/{args_dict['dataset']}_{model_name}_wrong_y.npy", np_wrong_y)
